이론 정리 페이지 : https://www.notion.so/leew-work/CH05-3-2cee0bd14001433cb5979d99e8ef6d21

In [23]:
from tensorflow import keras

### LSTM

keras에서 제공하는 IMDB를 사용할 예정  
- 표준 어휘를 사용해 영화 리뷰를 정수 배열로 변환한 데이터셋  
- 감정 분류하는 방법 학습 -> 새로운 텍스트에 대해 동일한 예 실행시켜 모델 검증 

문장들 : 감정이 긍정 <-> 부정을 나타내는 레이블과 함께 정수 배열 형태로 제공됨 

In [24]:
# keras 라이브러리, 그리고 imdb 데이터셋 로드
from keras.preprocessing import sequence
from keras.models import Sequential 
from keras.layers import Dense, Embedding 
from keras.layers import LSTM 
from keras.datasets import imdb

In [25]:
# 데이터셋에서 로드할 단어의 최대 수 지정 
max_features = 20000

# 문장의 최대 단어 수와 batch 크기 지정 
maxlen = 50
batch_size = 32

# 데이터 로드 
print('  Loading data...   ')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 문장의 패딩 
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

  Loading data...   
x_train shape: (25000, 50)
x_test shape: (25000, 50)


이제 데이터를 탐색하려고 한다.  
정수 배열을 보고 전체 문장을 얻기 위해 어휘를 사용한다. 

In [26]:
# 데이터 샘플 표시해 보기 
print("x_train 배열 샘플 = ", x_train[0])
print("y_train 배열 샘플 = ", y_train[0])

# 단어를 숫자로 변환하는 데 사용된 어휘 가져오기 
imdb_vocab = imdb.get_word_index()

# 단어를 숫자로 변환하는 데 사용된 어휘 가져오기 
# 이것은 단지 어휘 목록의 생김새를 이해하기 위한 목적이다. 
small_vocab = {key: value for key, value in imdb_vocab.items() if value < 20}
print('Vocabulary = ', small_vocab)

x_train 배열 샘플 =  [2071   56   26  141    6  194 7486   18    4  226   22   21  134  476
   26  480    5  144   30 5535   18   51   36   28  224   92   25  104
    4  226   65   16   38 1334   88   12   16  283    5   16 4472  113
  103   32   15   16 5345   19  178   32]
y_train 배열 샘플 =  1
Vocabulary =  {'with': 16, 'i': 10, 'as': 14, 'it': 9, 'is': 6, 'in': 8, 'but': 18, 'of': 4, 'this': 11, 'a': 3, 'for': 15, 'br': 7, 'the': 1, 'was': 13, 'and': 2, 'to': 5, 'film': 19, 'movie': 17, 'that': 12}


In [27]:
# 정수 배열에서 문장을 얻는 함수 
# 어휘에서 단어를 역조화(reverse look-up)한다. 
def get_original_text(int_arr):
    word_to_id = {k: (v+3) for k, v in imdb_vocab.items()}
    word_to_id['<PAD>'] = 0
    
    word_to_id['<START>'] = 1
    word_to_id['<UNK>'] = 2
    
    id_to_word = {value: key for key, value in word_to_id.items()}
    return ' '.join(id_to_word[id] for id in int_arr)

예측은 0과 1 사이의 값  
- 0에 가까운 값 : 감정이 긍정
- 1에 가까운 값 : 감정이 부정 

In [28]:
# 감정 배열 정의 
sentiment_labels = ['Positive', 'Negative'] # 0이 긍정이므로 0번째 index, 1이 부정이므로 1번째 index

print("================================================")
print("문장 및 감정 샘플")

# 훈련 데이터의 일부를 출력 
for i in range(5):
    print("훈련 문장 = ", get_original_text(x_train[i]))
    print("감정 = ", sentiment_labels[y_train[i]])
    print("---------------------------")

문장 및 감정 샘플
훈련 문장 =  grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
감정 =  Positive
---------------------------
훈련 문장 =  boobs and <UNK> taking away bodies and the gym still doesn't close for <UNK> all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then
감정 =  Negative
---------------------------
훈련 문장 =  must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how <UNK> this is to watch save yourself an hour a bit of your life
감정 =  Negative
---------------------------
훈련 문장 =  man to see a film that is true to scotland this one is probably unique if you maybe <UNK> o

이제 모델 구축하고 훈련  
앞의 Conv2D와 Dense Layer 대신 Embedding과 LSTM 계층 사용 

In [29]:
# 모델 구축 
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2))
model.add(Dense(1, activation='sigmoid')) # 긍정-부정 이진 분류

# 다른 최적화기들과 최적화기의 다른 설정을 사용해 본다. 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련 
model.fit(
    x_train, y_train,
    batch_size=batch_size, 
    epochs=2, 
    validation_data=(x_test, y_test)
)

score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

print("Test score:", score)
print("Accuracy: ", acc)

Epoch 1/2


782/782 ━━━━━━━━━━━━━━━━━━━━ 25s 30ms/step - accuracy: 0.7298 - loss: 0.5208 - val_accuracy: 0.8205 - val_loss: 0.3990
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.8825 - loss: 0.2839 - val_accuracy: 0.8102 - val_loss: 0.4209
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8059 - loss: 0.4271
Test score: 0.420929491519928
Accuracy:  0.8101599812507629


모델을 H5 형식인 imdb_nlp.h5 파일에 저장한다.  
저장된 모델 파일을 당장은 사용하지 않을 것이지만,  
8장 - 모델 배포 - 에서 사용할 예정  

예측은 0과 1 사이의 값  
- 0에 가까운 값 : 감정이 긍정
- 1에 가까운 값 : 감정이 부정 

In [30]:
# 문장에 대해 예측하기 
from tensorflow.keras.preprocessing.text import text_to_word_sequence

# 먼저 모델을 저장한다. 
model.save('imdb_nlp.h5')

# imdb 데이터셋에서 단어 색인을 가져오기 
word_index = imdb.get_word_index()

# 문서 정의 
my_sentence1 = 'really bad experience. amazingly bad.'
my_sentence2 = 'pretty awesome to see. very good work.'

# 모델을 사용해 감정을 예측하는 함수 정의 
def predict_sentiment(my_test):
    # 문장 토큰화
    word_sequence = text_to_word_sequence(my_test)
    
    # 빈 정수 시퀀스 생성 
    int_sequence = []
    
    # 문장의 각 단어에 대해 
    for w in word_sequence:
        # word_index(vocabulary)에서 정수를 읽어와 리스트에 추가 
        int_sequence.append(word_index[w])
        
    # 숫자 시퀀스를 모델 입력 크기에 맞게 패딩 
    sent_test = sequence.pad_sequences([int_sequence], maxlen=maxlen)
    
    # 모델을 사용한 예측 
    y_pred = model.predict(sent_test)
    
    return y_pred[0][0]

In [31]:
# 문장들에 대해 결과 출력 
print(
    'SENTENCE: ', my_sentence1, " : ", 
    predict_sentiment(my_sentence1), 
    ": SENTIMENT : ", 
    sentiment_labels[int(round(predict_sentiment(my_sentence1)))]
)

print(
    'SENTENCE: ', my_sentence2, " : ", 
    predict_sentiment(my_sentence2), 
    ": SENTIMENT : ", 
    sentiment_labels[int(round(predict_sentiment(my_sentence2)))]
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
SENTENCE:  really bad experience. amazingly bad.  :  0.7849308 : SENTIMENT :  Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
SENTENCE:  pretty awesome to see. very good work.  :  0.15248412 : SENTIMENT :  Negative


반대로 됐다,,,, 뭐지 